In [1]:
import pymongo
import pandas as pd
import numpy as np
import folium
import requests
from bs4 import BeautifulSoup as bs
from pandas.io.json import json_normalize
from pymongo import MongoClient
import re
import textract
import matplotlib.pyplot as plt
client=MongoClient()
db=client.DB

In [2]:
urljll = 'https://www.us.jll.com/en/research/cities-research/innovation-geographies'
urlnumbeo = 'https://www.numbeo.com/cost-of-living/rankings.jsp'

In [3]:
htmlinta = requests.get(urljll).content
soupinta = bs(htmlinta,'html.parser')
jllcity = [c for c in soupinta.find_all('ul',{'class':'list-cities'})]
most_innovative_city = jllcity[0].text.strip().split('\n')
most_talented_city = jllcity[1].text.strip().split('\n')
innovation_city = []
for y in most_innovative_city:
    t = [i for j in re.findall('([A-Z][a-z]+ ?[A-Z][a-z]+)|([A-Z][a-z]+)',y) for i in j if i != '']
    innovation_city.append(t[0])
talent_city = []
for y in most_talented_city:
    t = [i for j in re.findall('([A-Z][a-z]+ ?[A-Z][a-z]+)|([A-Z][a-z]+)',y) for i in j if i != '']
    talent_city.append(t[0])

In [4]:
htmlcost = requests.get(urlnumbeo).content
soupcost = bs(htmlcost,'html.parser')
citycostname = [e.text.split(',')[0] for e in soupcost.find_all('td',{'class':'cityOrCountryInIndicesTable'})]
citycostindexes = [e.text for e in soupcost.find_all('td',{'style':'text-align: right'})]
citycostindex = []
counter = 0
for i in range(len(citycostindexes)):
    try:
        citycostindex.append(float(citycostindexes[counter]))
        counter += 6
    except:
        break
citycost = dict(zip(citycostname,citycostindex))

In [5]:
cities_research = set(innovation_city+talent_city)

In [6]:
indexpais = pd.DataFrame(cities_research)
indexpais.columns = ['ciudad']

In [7]:
dicic = {r:s+1 for r,s in zip(innovation_city,range(len(innovation_city)))}

In [8]:
dictc = {r:s+1 for r,s in zip(talent_city,range(len(talent_city)))}

In [9]:
diccc =  {'Austin':{'latitude':30.267153,'longitude':-97.743057},
          'Berlin':{'latitude':52.520008,'longitude':13.404954},
          'Boston':{'latitude':42.360081,'longitude':-71.058884},
          'Paris':{'latitude':48.856613,'longitude':2.352222},
          'Shenzhen':{'latitude':22.543097,'longitude':114.057861},
          'San Jose':{'latitude':37.338207,'longitude':-121.886330},
          'Munich':{'latitude':48.135124,'longitude':11.581981},
          'Helsinki':{'latitude':60.169857,'longitude':24.938379},
          'Singapore':{'latitude':1.352083,'longitude':103.819839},
          'Zurich':{'latitude':47.376888,'longitude':8.541694},
          'Toronto':{'latitude':43.653225,'longitude':-79.383186},
          'Amsterdam':{'latitude':52.370216,'longitude':4.895168},
          'Edinburgh':{'latitude':55.953251,'longitude':-3.188267},
          'Los Angeles':{'latitude':34.052235,'longitude':-118.243683},
          'Shanghai':{'latitude':31.230391,'longitude':121.473701},
          'Sydney':{'latitude':-33.868820,'longitude':151.209290},
          'New York':{'latitude':40.712776,'longitude':-74.005974},
          'Stockholm':{'latitude':59.329323,'longitude':18.068581},
          'London':{'latitude':51.507351,'longitude':-0.127758},
          'Denver':{'latitude':39.739235,'longitude':-104.990250},
          'Melbourne':{'latitude':-37.813629,'longitude':144.963058},
          'Tokyo':{'latitude':35.689487,'longitude':139.691711},
          'San Francisco':{'latitude':37.774929,'longitude':-122.419418},
          'Seattle':{'latitude':47.606209,'longitude':-122.332069},
          'Oslo':{'latitude':59.913868,'longitude':10.752245},
          'Beijing':{'latitude':39.904202,'longitude':116.407394},
          'Seoul':{'latitude':37.5665,'longitude':126.9780},
          'Washington':{'latitude':38.9072,'longitude':-77.0369}}

In [10]:
incity = []
for t in indexpais.ciudad:
    if t in dicic:
        incity.append(dicic[t])
    else:
        incity.append(0)

talcity = []
for t in indexpais.ciudad:
    if t in dictc:
        talcity.append(dictc[t])
    else:
        talcity.append(0)
        
coordcity = []
for t in indexpais.ciudad:
    if t in diccc:
        coordcity.append(diccc[t])
    else:
        coordcity.append(0)
        
costcity = []
for t in indexpais.ciudad:
    if t in citycost:
        costcity.append(citycost[t])
    else:
        costcity.append(0)        
        
indexpais['rank_innov'] = incity
indexpais['rank_talento'] = talcity
indexpais['rank_costo'] = costcity
indexpais['coordenadas'] = coordcity

In [11]:
indexpais = indexpais.assign(integral=np.where(indexpais.rank_innov>0,25-indexpais.rank_innov*(15)/19,0)+
                          np.where(indexpais.rank_talento>0,25-indexpais.rank_talento*(15)/19,0)+
                          (2134.5/indexpais.rank_costo))

In [12]:
indexpais.set_index('ciudad',inplace=True)

In [13]:
indexpais.sort_values('integral',ascending=False)

,rank_innov,rank_talento,rank_costo,coordenadas,integral
ciudad,,,,,
London,5,1,59.97,"{'latitude': 51.507351, 'longitude': -0.127758}",80.855954
San Jose,6,4,60.16,"{'latitude': 37.338207, 'longitude': -121.88633}",77.585649
San Francisco,1,2,89.36,"{'latitude': 37.774929, 'longitude': -122.419418}",71.518105
Beijing,4,0,45.79,"{'latitude': 39.904202, 'longitude': 116.407394}",68.457087
Shenzhen,14,0,42.69,"{'latitude': 22.543097, 'longitude': 114.057861}",63.947368
Paris,7,8,85.42,"{'latitude': 48.856613, 'longitude': 2.352222}",63.146188
Boston,9,6,86.65,"{'latitude': 42.360081, 'longitude': -71.058884}",62.791478
Seattle,15,5,86.59,"{'latitude': 47.606209, 'longitude': -122.332069}",58.861179
Sydney,16,7,79.74,"{'latitude': -33.86882, 'longitude': 151.20929}",58.610352


In [14]:
z = db.db.find({'number_of_employees':{'$lte':1000},'founded_year':{'$gte':2000},'ipo':None,
                'offices.0.city':{'$exists': True },'offices.0.latitude':{'$exists': True }},
               {'_id':0,'name':1,'category_code':1,'number_of_employees':1,'offices':1})
zz = pd.DataFrame(z)
zz = zz.assign(city=zz.offices)
zz.city = zz.city.apply(lambda x : x[0]['city'])

In [15]:
def get_first(data):
    res=[]
    ofi=[]
    data=data['offices']
    for e in data:
        principal=None   # solo las tienen geodata
        if e[0]['latitude'] and e[0]['longitude']:
            principal={'type':'Point',
                       'coordinates':[
                           e[0]['longitude'],
                           e[0]['latitude']
                       ]}
            
        ofi.append(principal)
        
        res.append({
            'total_offices':len(e),
            'lat':e[0]['latitude'],
            'lng':e[0]['longitude'],
            'oficina_principal':principal
        })
    
    return ofi

In [16]:
zz = zz.assign(geo=get_first(zz))

In [17]:
zz = zz.assign(lat = zz.offices.apply(lambda x : x[0]['latitude']))

In [18]:
zz = zz.assign(long = zz.offices.apply(lambda x : x[0]['longitude']))

In [19]:
zz

,name,category_code,number_of_employees,offices,city,geo,lat,long
0,Wetpaint,web,47,"[{'description': '', 'address1': '710 - 2nd Av...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253
1,Digg,news,60,"[{'description': None, 'address1': '135 Missis...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523
2,Geni,web,18,"[{'description': 'Headquarters', 'address1': '...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064
3,Scribd,news,50,"[{'description': 'HQ', 'address1': '539 Bryant...",San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052
4,MeetMoi,social,15,"[{'description': None, 'address1': '', 'addres...",New York City,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506
...,...,...,...,...,...,...,...,...
6005,Getyoo,mobile,10,"[{'description': 'HQ', 'address1': 'Rue des Ta...",Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5...",50.839970,4.346472
6006,OfficialVirtualDJ,music,102,"[{'description': 'Created in London', 'address...",London,None,NaN,NaN
6007,Eazeeloans,web,25,"[{'description': 'Apex Finance & Marketing', '...",Mumbai,None,NaN,NaN
6008,DocASAP,web,7,"[{'description': '', 'address1': '115 5th Ave'...",New York,None,NaN,NaN


In [25]:
choice_ciudad = input()

San Francisco


In [26]:
mapa=folium.Map(location=[indexpais.loc[choice_ciudad].coordenadas['latitude'],
                          indexpais.loc[choice_ciudad].coordenadas['longitude']],
                          tiles='stamentoner', zoom_start=12)
folium.CircleMarker([indexpais.loc[choice_ciudad].coordenadas['latitude'],
                     indexpais.loc[choice_ciudad].coordenadas['longitude']],
                     radius=10, icon=folium.Icon()).add_to(mapa)

In [27]:
startupfilter = zz[zz.city==choice_ciudad]
startupfilter

,name,category_code,number_of_employees,offices,city,geo,lat,long
1,Digg,news,60,"[{'description': None, 'address1': '135 Missis...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523
3,Scribd,news,50,"[{'description': 'HQ', 'address1': '539 Bryant...",San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052
7,Powerset,search,60,"[{'description': None, 'address1': '475 Branna...",San Francisco,"{'type': 'Point', 'coordinates': [-122.395289,...",37.778613,-122.395289
8,Technorati,advertising,35,"[{'description': '', 'address1': '360 Post St....",San Francisco,"{'type': 'Point', 'coordinates': [-122.393041,...",37.779558,-122.393041
12,Kyte,games_video,40,"[{'description': None, 'address1': '442 Post S...",San Francisco,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173
...,...,...,...,...,...,...,...,...
5758,Context Optional,other,45,"[{'description': '', 'address1': '30 Maiden La...",San Francisco,None,NaN,NaN
5800,ZoomSystems,hardware,100,"[{'description': 'Corporate Headquarters', 'ad...",San Francisco,"{'type': 'Point', 'coordinates': [-122.391061,...",37.781689,-122.391061
5854,TwtBks,web,3,"[{'description': 'TwtBks', 'address1': '442 Sh...",San Francisco,"{'type': 'Point', 'coordinates': [-122.4160008...",37.761102,-122.416001
5919,Fuze Box,software,120,"[{'description': 'Headquarters', 'address1': '...",San Francisco,None,NaN,NaN


In [28]:
for i in range(len(startupfilter.lat)):
    try:
        folium.Marker([startupfilter.lat.iloc[i],startupfilter.long.iloc[i]],
                      popup = (startupfilter.name.iloc[i],startupfilter.category_code.iloc[i]),
                      tooltip='haz click aqui').add_to(mapa)
    except:
        pass
mapa

In [29]:
comp = startupfilter.groupby('category_code').agg('count')
comp['name'].sort_values(ascending = False)

category_code
web                 91
software            34
games_video         31
mobile              29
enterprise          28
advertising         27
ecommerce           16
search              12
other               12
hardware            10
consulting           9
analytics            8
cleantech            7
finance              6
public_relations     6
network_hosting      6
social               4
news                 4
security             4
music                3
travel               2
health               1
legal                1
sports               1
education            1
nonprofit            1
design               1
messaging            1
Name: name, dtype: int64